In [1]:
# imports
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
# loading data
data = pd.read_csv('data/OnlineNewsPopularityCleanRegression.csv')

# 80/20 split between training data and test data
X_train, X_test, Y_train, Y_test = train_test_split(data[["n_tokens_title", "n_tokens_content", "n_unique_tokens", "n_non_stop_words", "n_non_stop_unique_tokens", "num_hrefs", "num_self_hrefs", "num_imgs", "num_videos", "average_token_length", "num_keywords", "data_channel_is_lifestyle", "data_channel_is_entertainment", "data_channel_is_bus", "data_channel_is_socmed", "data_channel_is_tech", "data_channel_is_world", "kw_min_min", "kw_max_min", "kw_avg_min", "kw_min_max", "kw_max_max", "kw_avg_max", "kw_min_avg", "kw_max_avg", "kw_avg_avg", "self_reference_min_shares", "self_reference_max_shares", "self_reference_avg_sharess", "weekday_is_monday", "weekday_is_tuesday", "weekday_is_wednesday", "weekday_is_thursday", "weekday_is_friday", "weekday_is_saturday", "weekday_is_sunday", "is_weekend", "LDA_00", "LDA_01", "LDA_02", "LDA_03", "LDA_04", "global_subjectivity", "global_sentiment_polarity", "global_rate_positive_words", "global_rate_negative_words", "rate_positive_words", "rate_negative_words", "avg_positive_polarity", "min_positive_polarity", "max_positive_polarity", "avg_negative_polarity", "min_negative_polarity", "max_negative_polarity", "title_subjectivity", "title_sentiment_polarity", "abs_title_subjectivity", "abs_title_sentiment_polarity"]], 
                                                    data['shares'], test_size=0.2, random_state=112)

# different regression models to test
models = [LinearRegression(n_jobs=-1),
          Ridge(),
          Lasso(),
          DecisionTreeRegressor(),
          RandomForestRegressor(n_jobs=-1),
          GradientBoostingRegressor(),
          KNeighborsRegressor(n_jobs=-1),
        ]

model_names = ['Linear Regression', 'Ridge', 'Lasso', 'Decision Tree', 'Random Forest', 'Gradient Boosting', 'K-NN']

In [3]:
def print_results(results):
    print(f'BEST PARAMS: {results.best_params_}\n')

    means = sorted(results.cv_results_['mean_test_score'], reverse=True)
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print(f'{round(mean,3)} (+/- {round(std * 2, 3)}) for {params}')

In [4]:
# tuning linear regression
print(f"RUNNING {model_names[0]}")
dtparams = {
    'fit_intercept': [False, True],
    'positive': [False, True]
}
cv = GridSearchCV(models[0], dtparams, cv=5, n_jobs=32)
cv.fit(X_train, Y_train.values.ravel())
print_results(cv)

RUNNING Linear Regression
BEST PARAMS: {'fit_intercept': False, 'positive': True}

0.018 (+/- 0.511) for {'fit_intercept': False, 'positive': False}
-0.108 (+/- 0.016) for {'fit_intercept': False, 'positive': True}
-0.109 (+/- 0.511) for {'fit_intercept': True, 'positive': False}
-6.615383143997146e+16 (+/- 2.6461532575988582e+17) for {'fit_intercept': True, 'positive': True}


In [5]:
# tuning Ridge
print(f"RUNNING {model_names[1]}")
dtparams = {
    'alpha': [0.5, 1],
    'fit_intercept': [False, True],
    'positive': [False, True],
    'max_iter': [1000, 5000, 15000],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']
}
cv = GridSearchCV(models[1], dtparams, cv=5, n_jobs=6)
cv.fit(X_train, Y_train.values.ravel())
print_results(cv)

RUNNING Ridge


In [11]:
# tuning Lasso
print(f"RUNNING {model_names[2]}")
dtparams = {
    'alpha': [0.5, 1],
    'fit_intercept': [False, True],
    'positive': [False, True],
    'max_iter': [1000, 5000, 15000],
    'selection': ['cyclic', 'random']
}
cv = GridSearchCV(models[2], dtparams, cv=5, n_jobs=32)
cv.fit(X_train, Y_train.values.ravel())
print_results(cv)

BEST PARAMS: {'alpha': 0.5, 'fit_intercept': True, 'max_iter': 1000, 'positive': False, 'selection': 'cyclic'}

0.07 (+/- 0.023) for {'alpha': 0.5, 'fit_intercept': False, 'max_iter': 1000, 'positive': False, 'selection': 'cyclic'}
0.07 (+/- 0.023) for {'alpha': 0.5, 'fit_intercept': False, 'max_iter': 1000, 'positive': False, 'selection': 'random'}
0.07 (+/- 0.025) for {'alpha': 0.5, 'fit_intercept': False, 'max_iter': 1000, 'positive': True, 'selection': 'cyclic'}
0.07 (+/- 0.025) for {'alpha': 0.5, 'fit_intercept': False, 'max_iter': 1000, 'positive': True, 'selection': 'random'}
0.07 (+/- 0.023) for {'alpha': 0.5, 'fit_intercept': False, 'max_iter': 5000, 'positive': False, 'selection': 'cyclic'}
0.07 (+/- 0.023) for {'alpha': 0.5, 'fit_intercept': False, 'max_iter': 5000, 'positive': False, 'selection': 'random'}
0.07 (+/- 0.025) for {'alpha': 0.5, 'fit_intercept': False, 'max_iter': 5000, 'positive': True, 'selection': 'cyclic'}
0.07 (+/- 0.025) for {'alpha': 0.5, 'fit_intercept'

In [6]:
# tuning Decision Tree
print(f"RUNNING {model_names[3]}")
dtparams = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'splitter': ['best', 'random'],
    'max_features': [None, 'auto', 'sqrt', 'log2']
}
cv = GridSearchCV(models[3], dtparams, cv=5, n_jobs=8)
cv.fit(X_train, Y_train.values.ravel())
print_results(cv)

NameError: name 'print_results' is not defined